In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# -----------------------
# Configuración general
# -----------------------
N_FILAS = 5000
SEED = 42
RUTA_SALIDA = os.path.join("..", "data_sintetica")
NOMBRE_BASE = "fundicion_ilo"

np.random.seed(SEED)
os.makedirs(RUTA_SALIDA, exist_ok=True)

# -----------------------
# Catálogos / helpers
# -----------------------
proveedores = [f"Proveedor_{i:02d}" for i in range(1, 21)]
puertos = ["Matarani", "Callao", "Paita", "Arica", "Antofagasta", "Mejillones"]
transportes = ["barco", "tren", "camion"]

# Fechas últimos 18 meses (más densidad reciente)
hoy = datetime.today().date()
inicio = hoy - timedelta(days=int(18 * 30))
fechas = pd.to_datetime(
    np.random.choice(pd.date_range(inicio, hoy, freq="D"), size=N_FILAS,
                     p=None)  # simple; se puede ponderar por cercanía
)

# Precio LME con leve tendencia + ruido
t = (fechas - fechas.min()).days.values
precio_base = 3.8 + 0.0007 * t  # tendencia suave
precio_lme = precio_base + np.random.normal(0, 0.2, size=N_FILAS)
precio_lme = np.clip(precio_lme, 3.2, 5.2)

# Mineral (impurezas 8%)
es_impuro = np.random.rand(N_FILAS) < 0.08
mineral = np.where(es_impuro, "cobre_impurezas", "cobre")

# Peso de lote (log-normal controlado)
peso_tn = np.random.lognormal(mean=np.log(20000), sigma=0.45, size=N_FILAS)
peso_tn = np.clip(peso_tn, 8000, 40000)

# Ley y humedad
ley_cu = np.random.normal(27.0, 3.5, size=N_FILAS)  # %
ley_cu = np.clip(ley_cu, 18.0, 36.0)

humedad = np.random.normal(8.5, 1.2, size=N_FILAS)  # %
humedad = np.clip(humedad, 6.0, 11.5)

# Recuperación
recuperacion = np.random.normal(96.0, 1.0, size=N_FILAS)
recuperacion = np.clip(recuperacion, 93.0, 98.5)

# Cargos y costos (dependen de variables)
tc_usd_tn = np.random.normal(95, 15, size=N_FILAS)           # 70–130
tc_usd_tn = np.clip(tc_usd_tn, 70, 130)

# Costo logístico según puerto y transporte
puerto = np.random.choice(puertos, size=N_FILAS)
transporte = np.random.choice(transportes, size=N_FILAS, p=[0.7, 0.2, 0.1])

base_log = np.where(np.isin(puerto, ["Arica", "Antofagasta", "Mejillones"]), 35, 28)
ajuste_transporte = np.select(
    [transporte=="barco", transporte=="tren", transporte=="camion"],
    [0, -5, +10], default=0
)
costo_logistico = base_log + ajuste_transporte + np.random.normal(0, 3, size=N_FILAS)
costo_logistico = np.clip(costo_logistico, 18, 48)

# Penalidad por As (más alta si impurezas o humedad alta)
penalidad = np.where(es_impuro, np.random.normal(25, 7, N_FILAS), np.random.normal(8, 5, N_FILAS))
penalidad += np.where(humedad > 9.5, (humedad - 9.5) * 4.0, 0.0)
penalidad = np.clip(penalidad, 0, 45)

# Ingreso neto (derivado)
LB_POR_TON = 2204.62
contenido_sec_lb = peso_tn * LB_POR_TON * (ley_cu / 100.0)
pagable_lb = contenido_sec_lb * (recuperacion / 100.0) * 0.965
ingreso_bruto = pagable_lb * precio_lme
costos = (tc_usd_tn + costo_logistico + penalidad) * peso_tn
ingreso_neto = ingreso_bruto - costos

# Identificador de lote
id_lote = [f"L-{d.strftime('%Y%m')}-{i:05d}" for i, d in enumerate(fechas, start=1)]

# Construcción del DataFrame (15 variables)
df = pd.DataFrame({
    "id_lote": id_lote,
    "fecha_llegada": fechas.normalize(),
    "proveedor": np.random.choice(proveedores, size=N_FILAS),
    "puerto_origen": puerto,
    "mineral": mineral,
    "metodo_transporte": transporte,
    "peso_tn": np.round(peso_tn, 2),
    "ley_cu_pct": np.round(ley_cu, 2),
    "humedad_pct": np.round(humedad, 2),
    "recuperacion_pct": np.round(recuperacion, 2),
    "precio_lme_usd_lb": np.round(precio_lme, 3),
    "tc_usd_tn": np.round(tc_usd_tn, 2),
    "costo_logistico_usd_tn": np.round(costo_logistico, 2),
    "penalidad_as_usd_tn": np.round(penalidad, 2),
    "ingreso_neto_usd": np.round(ingreso_neto, 2),
})

# Guardado
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
pkl_path = os.path.join(RUTA_SALIDA, f"{NOMBRE_BASE}_{timestamp}.pkl")
csv_path = os.path.join(RUTA_SALIDA, f"{NOMBRE_BASE}_{timestamp}.csv")

df.to_pickle(pkl_path)
df.to_csv(csv_path, index=False, encoding="utf-8")

print(f"✅ Dataset creado con {len(df):,} filas y 15 columnas")
print(f"🧺 PKL: {pkl_path}")
print(f"📄 CSV: {csv_path}")

✅ Dataset creado con 5,000 filas y 15 columnas
🧺 PKL: ..\data_sintetica\fundicion_ilo_20250814_2015.pkl
📄 CSV: ..\data_sintetica\fundicion_ilo_20250814_2015.csv
